In [2]:
import struct
import array
import zlib
import gzip
import numpy as np
import math
import xarray as xr


In [7]:
def read_gdb_file(file_path):
    with open(file_path, 'rb') as file:

        header = file.read(24)
        #file_code, = struct.unpack('<i', header[:4])
        file_description = header[4:24].decode('utf-8').strip('\x00')
        
        magic_number = struct.unpack("<I", file.read(4))[0]
        
        #print(f"File Code: {file_code}")
        print(f"File Description: {file_description}")
        
        print(f"Magic Number: {magic_number}")

# Use the function
read_gdb_file('data/DB_1116.gdb')

File Description: 
Magic Number: 50


In [35]:
def read_geosoft_gdb(file_path):
    with open(file_path, 'rb') as file:
        # Read the header (example: assuming the header is 256 bytes)
        header = file.read(256)
        print(header[10:13])
        print(header)
        
        # Unpack the header (example: assuming the header contains an integer and a float)
        #header_data = struct.unpack('if', header[:8])
        header_data = struct.unpack('ssss', header[:4])
        print("Header Data:", header_data)
        
        # Read the rest of the file (example: assuming the rest of the file contains records of 16 bytes each)
        while True:
            record = file.read(16)
            break
            if not record:
                break
            # Unpack the record (example: assuming each record contains two integers and two floats)
            record_data = struct.unpack('iiff', record)
            print("Record Data:", record_data)

# Example usage
read_geosoft_gdb('data/DB_1116.gdb') 

b'\x02\x10\x08'
b'!CBD\x00\x00\x00\x00\x00\x00\x02\x10\x08\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x002\x00\x00\x00^\x01\x00\x00d\x00\x00\x00\xc8\x00\x00\x00\n\x00\x00\x00\xdc(\x00\x00\x10\'\x00\x00n(\x00\x00x(\x00\x00x(\x00\x00b\x02\x00\x00\x00\x00\x00\x00^\x01\x00\x00&\x02\x00\x00X\x02\x00\x00^\x01\x00\x00\xc8\x00\x00\x002\x00\x00\x00\n\x00\x00\x00\x00\x04\x00\x00@\'\x02\x00\x8a\x00\x00\x00"=\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Header Data: (b'!', b'C', b'B', b

In [33]:
def search_for_patterns(file_path):
    # Known compression signatures and their descriptions
    patterns = {
        b'\x78\x9C': 'zlib (deflate)',
        b'\x78\xDA': 'zlib (deflate)',
        b'\x1F\x8B': 'GZIP',
        b'\x42\x5A\x68': 'BZIP2',
        b'\xFD\x37\x7A\x58\x5A\x00': 'LZMA',
        b'\x50\x4B\x03\x04': 'ZIP'
    }

    with open(file_path, 'rb') as file:
        data = file.read()
    
    # Search for each pattern in the binary data
    for pattern, description in patterns.items():
        offset = data.find(pattern)
        if offset != -1:
            print(f"Found {description} compression marker at offset {offset:08X}")
        else:
            print(f"{description} marker not found")

# Replace with the path to your binary file
search_for_patterns('data/DB_1116.gdb')


Found zlib (deflate) compression marker at offset 0003334A
Found zlib (deflate) compression marker at offset 0002E59C
Found GZIP compression marker at offset 000ECF32
BZIP2 marker not found
LZMA marker not found
ZIP marker not found


In [37]:

def decompress_zlib(data, offset):
    try:
        # Attempt decompression starting from the given offset
        decompressed_data = zlib.decompress(data[offset:])
        print(f"Decompressed zlib data from offset {offset:08X} (first 100 bytes): {decompressed_data[:100]}")
        return decompressed_data
    except zlib.error as e:
        print(f"Failed to decompress zlib data at offset {offset:08X}: {e}")
        return None

def decompress_gzip(data, offset):
    try:
        # Gzip requires wrapping the data in a file-like object
        decompressed_data = gzip.decompress(data[offset:])
        print(f"Decompressed GZIP data from offset {offset:08X} (first 100 bytes): {decompressed_data[:100]}")
        return decompressed_data
    except gzip.BadGzipFile as e:
        print(f"Failed to decompress GZIP data at offset {offset:08X}: {e}")
        return None

def read_and_decompress(file_path):
    with open(file_path, 'rb') as file:
        data = file.read()
    
    # Found offsets for zlib and GZIP markers
    zlib_offsets = [0x3334A, 0x2E59C]
    gzip_offsets = [0xECF32]
    
    # Process zlib sections
    for offset in zlib_offsets:
        print(f"\nAttempting to decompress zlib data at offset {offset:08X}")
        decompressed_data = decompress_zlib(data, offset)
    
    # Process GZIP sections
    for offset in gzip_offsets:
        print(f"\nAttempting to decompress GZIP data at offset {offset:08X}")
        decompressed_data = decompress_gzip(data, offset)

# Replace with the path to your binary file
read_and_decompress('data/DB_1116.gdb')



Attempting to decompress zlib data at offset 0003334A
Failed to decompress zlib data at offset 0003334A: Error -3 while decompressing data: invalid block type

Attempting to decompress zlib data at offset 0002E59C
Failed to decompress zlib data at offset 0002E59C: Error -3 while decompressing data: invalid code lengths set

Attempting to decompress GZIP data at offset 000ECF32
Failed to decompress GZIP data at offset 000ECF32: Unknown compression method


In [38]:
def try_partial_zlib_decompression(data, offset, length=1024):
    try:
        # Attempt to decompress a portion of the data starting at the given offset
        decompressed_data = zlib.decompress(data[offset:offset + length])
        print(f"Partial zlib decompression successful at offset {offset:08X} (first 100 bytes): {decompressed_data[:100]}")
        return decompressed_data
    except zlib.error as e:
        print(f"Partial zlib decompression failed at offset {offset:08X}: {e}")
        return None

def try_partial_gzip_decompression(data, offset, length=1024):
    try:
        # Attempt to decompress a portion of the data starting at the given offset
        decompressed_data = gzip.decompress(data[offset:offset + length])
        print(f"Partial GZIP decompression successful at offset {offset:08X} (first 100 bytes): {decompressed_data[:100]}")
        return decompressed_data
    except gzip.BadGzipFile as e:
        print(f"Partial GZIP decompression failed at offset {offset:08X}: {e}")
        return None

def read_and_attempt_partial_decompression(file_path):
    # Step 1: Read the entire file into memory
    with open(file_path, 'rb') as file:
        data = file.read()
    
    # Step 2: Offsets identified for zlib and GZIP markers
    zlib_offsets = [0x3334A, 0x2E59C]
    gzip_offsets = [0xECF32]
    
    # Step 3: Try partial decompression for each zlib offset
    for i, offset in enumerate(zlib_offsets, start=1):
        print(f"\nAttempting partial zlib decompression at offset {offset:08X}")
        try_partial_zlib_decompression(data, offset)
    
    # Step 4: Try partial decompression for each GZIP offset
    for i, offset in enumerate(gzip_offsets, start=1):
        print(f"\nAttempting partial GZIP decompression at offset {offset:08X}")
        try_partial_gzip_decompression(data, offset)

# Replace with the path to your binary file
read_and_attempt_partial_decompression('data/DB_1116.gdb')



Attempting partial zlib decompression at offset 0003334A
Partial zlib decompression failed at offset 0003334A: Error -3 while decompressing data: invalid block type

Attempting partial zlib decompression at offset 0002E59C
Partial zlib decompression failed at offset 0002E59C: Error -3 while decompressing data: invalid code lengths set

Attempting partial GZIP decompression at offset 000ECF32
Partial GZIP decompression failed at offset 000ECF32: Unknown compression method


In [68]:
def extract_segment(data, offset, length=500):
    # Extract data around the offset to inspect it more closely
    #start = max(0, offset - 100)  # Start 100 bytes before offset
    start = max(0, offset - 0)  # Start 100 bytes before offset
    end = offset + length          # Go 500 bytes past offset
    segment = data[start:end]
    
    # Save to file for external inspection
    output_file = f'segment_{offset:08X}.bin'
    with open(output_file, 'wb') as f:
        f.write(segment)
    
    print(f"Saved segment around offset {offset:08X} to {output_file}")
    return segment

# Example of use
with open('data/DB_1116.gdb', 'rb') as file:
    data = file.read()
    offsets = [0x3334A, 0x2E59C, 0xECF32]
    for offset in offsets:
        extract_segment(data, offset)


Saved segment around offset 0003334A to segment_0003334A.bin
Saved segment around offset 0002E59C to segment_0002E59C.bin
Saved segment around offset 000ECF32 to segment_000ECF32.bin


In [41]:
# Replace with the path to your binary file
read_and_attempt_partial_decompression('segment_000ECF32.bin')

read_and_attempt_partial_decompression('segment_0002E59C.bin')

read_and_attempt_partial_decompression('segment_0003334A.bin')



Attempting partial zlib decompression at offset 0003334A
Partial zlib decompression failed at offset 0003334A: Error -5 while decompressing data: incomplete or truncated stream

Attempting partial zlib decompression at offset 0002E59C
Partial zlib decompression failed at offset 0002E59C: Error -5 while decompressing data: incomplete or truncated stream

Attempting partial GZIP decompression at offset 000ECF32
Partial GZIP decompression successful at offset 000ECF32 (first 100 bytes): b''

Attempting partial zlib decompression at offset 0003334A
Partial zlib decompression failed at offset 0003334A: Error -5 while decompressing data: incomplete or truncated stream

Attempting partial zlib decompression at offset 0002E59C
Partial zlib decompression failed at offset 0002E59C: Error -5 while decompressing data: incomplete or truncated stream

Attempting partial GZIP decompression at offset 000ECF32
Partial GZIP decompression successful at offset 000ECF32 (first 100 bytes): b''

Attempting 

In [50]:
def read_and_attempt_partial_gzip_decompression(file_path):
    # Step 1: Read the entire file into memory
    with open(file_path, 'rb') as file:
        data = file.read()
    
    gzip_offsets = [0x1F9F0]
    gzip_offsets = [0x23420]
    
    # Step 4: Try partial decompression for each GZIP offset
    for i, offset in enumerate(gzip_offsets, start=100):
        print(f"\nAttempting partial GZIP decompression at offset {offset:08X}")
        decomp = try_partial_gzip_decompression(data, offset)
        return decomp

# Replace with the path to your binary file
read_and_attempt_partial_gzip_decompression('data/DB_1116.gdb')




Attempting partial GZIP decompression at offset 00023420
Partial GZIP decompression failed at offset 00023420: Not a gzipped file (b'\x00\x00')


In [46]:
with open('data/DB_1116.gdb', 'rb') as file:
    data = file.read()
    offsets = [0x234E0]
    for offset in offsets:
        extract_segment(data, offset)


Saved segment around offset 000234E0 to segment_000234E0.bin


In [65]:
def try_partial_gzip_decompression(data, offset, length=1024):
    try:
        print(data)
        # Attempt to decompress a portion of the data starting at the given offset
        decompressed_data = gzip.decompress(data[offset:offset + length])
        print("DD:",decompressed_data)
        print(f"Partial GZIP decompression successful at offset {offset:08X} (first 100 bytes): {decompressed_data[:100]}")
        return decompressed_data
    except gzip.BadGzipFile as e:
        print(f"Partial GZIP decompression failed at offset {offset:08X}: {e}")
        return None

def read_and_attempt_partial_gzip_decompression(file_path):
    # Step 1: Read the entire file into memory
    with open(file_path, 'rb') as file:
        data = file.read()
    
    gzip_offsets = [0x1F9F0]
    gzip_offsets = [0x00000]
    
    # Step 4: Try partial decompression for each GZIP offset
    for i, offset in enumerate(gzip_offsets):
        print(f"\nAttempting partial GZIP decompression at offset {offset:08X}")
        decomp = try_partial_gzip_decompression(data, offset)
        return decomp


In [69]:
read_and_attempt_partial_gzip_decompression('segment_000ECF32.bin')


Attempting partial GZIP decompression at offset 00000000
b'\x1f\x8b\xc8\x1cA+c\xa2\\\xa6\x02\x08\xd1\xe5\x08\x82\x05\x9a\xc1\x02\x08\xb6hi\xd7\xdc\x02\x08{\xeb\xcc\x14\xf8\x02\x08\x00 Sn0R\x13\xc9\x1cA\xc2\xf1\x93\x8f.\x02\x08\x16u\x08\x82\xf7\xccI\x02\x08\xa9\xf8Z\ne\x02\x08\x91|\xbeG\x80\x02\x08 \x08\x8d\x00"\x85\x9b\x02\x08T\xf0\xc2\xcc\xb6\x02\x08`t&\n\x82\x00\xd2\x02\x08\x93d\xc7Q\xed\x02\x08g\xe9*\x8f\x08\xca\x1cA \x08\xc4\xd9\xcb\xd6#\x02\x08\xdd\xc9l\x1e?\x02\x08\xf3%Kp\x82 Z\x02\x08N\x16\xec\xb7u\x02\x08\xc1r\xca\t\x91\x02\x08\x1e\xcf\x08\x82\xa8[\xac\x02\x08\xaa+\x87\xad\xc7\x02\x08"\x88e\xff\xe2\x02\x08 \x00\xfa\xe4CQ\xfe\x02\x08\xf0\xad_\xad\x19\xcb\x1cA\xd5v\x08\x82{\t5\x02\x08\xdb?\x97eP\x02\x08+\t\xb3\xc1k\x02\x08 \x08H>\x0c(\x87\x02\x08\x85se\x8e\xa2\x02\x08\x17\xa9\xbe\xf4\x82 \xbd\x02\x08p\xde\x17[\xd9\x02\x08\xcd\x7f\xae\xcb\xf4\x02\x08~!\x00\x08E<\x10\xcc\x1cA\xec?\xa8y,\x02\x08\x89\xe1>\xea\x82 G\x02\x08\xb4\xee\x12ec\x02\x08\x14\xfc\xe6\xdf~\x02\x08\xd8\t\x08\x82

In [13]:

def calculate_entropy(data):
    # Calculate the probability distribution of each byte value
    probabilities = np.unique(data, return_counts=True)
    probabilities = probabilities[1] / len(data)

    # Calculate the entropy
    entropy = -np.sum(probabilities * np.log2(probabilities))

    return entropy

# Load the binary data
data = np.fromfile('data/DB_1116.gdb')

# Calculate the entropy
entropy = calculate_entropy(data)
print('Entropy:', entropy)

# Define the block size
block_size = 256

# Calculate the entropy for each block
for i in range(0, len(data), block_size):
    block = data[i:i+block_size]
    entropy = calculate_entropy(block)
    print (i, round(entropy,1))
    #print(f'Block {i:block_size}, Entropy = {entropy}')


Entropy: 19.801949066615972
0 3.1
256 3.0
512 3.0
768 3.0
1024 2.9
1280 2.8
1536 3.0
1792 3.0
2048 3.0
2304 3.0
2560 2.2
2816 -0.0
3072 -0.0
3328 -0.0
3584 -0.0
3840 -0.0
4096 -0.0
4352 -0.0
4608 -0.0
4864 -0.0
5120 -0.0
5376 -0.0
5632 -0.0
5888 -0.0
6144 -0.0
6400 -0.0
6656 -0.0
6912 -0.0
7168 -0.0
7424 0.6
7680 2.1
7936 1.3
8192 5.4
8448 6.0
8704 6.0
8960 6.0
9216 6.0
9472 6.0
9728 6.0
9984 6.0
10240 6.0
10496 6.0
10752 6.0
11008 6.0
11264 6.0
11520 6.0
11776 6.0
12032 6.0
12288 6.0
12544 6.0
12800 6.0
13056 6.0
13312 4.8
13568 2.4
13824 2.4
14080 2.4
14336 2.4
14592 7.2
14848 8.0
15104 8.0
15360 8.0
15616 8.0
15872 7.1
16128 6.9
16384 7.8
16640 3.4
16896 7.0
17152 5.8
17408 5.0
17664 1.0
17920 4.8
18176 8.0
18432 8.0
18688 8.0
18944 8.0
19200 8.0
19456 8.0
19712 8.0
19968 8.0
20224 6.4
20480 8.0
20736 8.0
20992 8.0
21248 8.0
21504 8.0
21760 7.9
22016 8.0
22272 7.1
22528 8.0
22784 8.0
23040 8.0
23296 8.0
23552 8.0
23808 8.0
24064 8.0
24320 8.0
24576 8.0
24832 8.0
25088 4.8
25344 8.0
